In [1]:
from torchtext.data.utils import get_tokenizer
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
import itertools
from torch.nn import Transformer


In [2]:
iwslt_dataset = load_dataset('iwslt2017', 'iwslt2017-en-de')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.backends.mps.is_available():
    device = 'mps'  # For Apple Silicon GPUs

print('DEVICE:', device)


BATCH_SIZE = 1


DEVICE: mps


In [3]:
# Initialize tokenizers
tokenizer_en = get_tokenizer('spacy', language='en_core_web_sm')
tokenizer_de = get_tokenizer('spacy', language='de_core_news_sm')

def tokenize(batch):
    # Access nested 'translation' and then language specific texts
    en_texts = [item['en'] for item in batch['translation']]
    de_texts = [item['de'] for item in batch['translation']]
    # Apply tokenization and ensure each text is a list of tokens
    batch['tokenized_en'] = [list(map(str, tokenizer_en(text))) for text in en_texts]
    batch['tokenized_de'] = [list(map(str, tokenizer_de(text))) for text in de_texts]
    return batch

# Apply the tokenization function
iwslt_dataset = iwslt_dataset.map(tokenize, batched=True, batch_size=1000, num_proc=4)

# Extract tokenized data
en_tokenized_texts = iwslt_dataset['train']['tokenized_en']
de_tokenized_texts = iwslt_dataset['train']['tokenized_de']



In [4]:
def build_vocab(tokenized_texts, min_freq=1):
    # Flatten the list of tokens
    all_tokens = list(itertools.chain.from_iterable(tokenized_texts))
    
    # Count all tokens
    token_freqs = Counter(all_tokens)
    
    # Remove tokens below a certain frequency threshold
    vocab = {token: idx + 1 for idx, (token, freq) in enumerate(token_freqs.items()) if freq >= min_freq}
    vocab['<pad>'] = 0  # Pad token at index 0
    vocab['<unk>'] = len(vocab)  # Unknown token at the last index
    return vocab

# Build vocabularies
EN_VOCAB = build_vocab(en_tokenized_texts)
DE_VOCAB = build_vocab(de_tokenized_texts)


In [5]:
def collate_fn(batch):
    en_batch = [item['tokenized_en'] for item in batch]
    de_batch = [['<sos>'] + item['tokenized_de'] + ['<eos>'] for item in batch]
    
    en_indices = [[EN_VOCAB.get(token, EN_VOCAB['<unk>']) for token in sentence] for sentence in en_batch]
    de_indices = [[DE_VOCAB.get(token, DE_VOCAB['<unk>']) for token in sentence] for sentence in de_batch]
    
    en_tensor = pad_sequence([torch.tensor(seq) for seq in en_indices], padding_value=EN_VOCAB['<pad>'], batch_first=True)
    de_tensor = pad_sequence([torch.tensor(seq) for seq in de_indices], padding_value=DE_VOCAB['<pad>'], batch_first=True)
    
    return {'en': en_tensor.to(device), 'de': de_tensor.to(device)}



In [6]:
# Initialize embedding layers
d_model = 512

en_embedding = torch.nn.Embedding(len(EN_VOCAB), d_model).to(device)
de_embedding = torch.nn.Embedding(len(DE_VOCAB), d_model).to(device)


model = Transformer(
    d_model=d_model,
    nhead=8,
    num_encoder_layers=6,
    num_decoder_layers=6,
    dim_feedforward=2048,
    dropout=0.1,    
).to(device)

output_projection = torch.nn.Linear(d_model, len(DE_VOCAB)).to(device)

/opt/homebrew/Caskroom/miniconda/base/envs/torchtext/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


### Counting Dimensions of Feed-Forward Network

In [7]:
def print_feedforward_dimensions(transformer):
    # Print dimensions for the encoder feed-forward network
    if transformer.encoder is not None and len(transformer.encoder.layers) > 0:
        encoder_ffn_layer1 = transformer.encoder.layers[0].linear1
        encoder_ffn_layer2 = transformer.encoder.layers[0].linear2
        print("Encoder Feedforward Network:")
        print(f"  Layer 1: Input Features = {encoder_ffn_layer1.in_features}, Output Features = {encoder_ffn_layer1.out_features}")
        print(f"  Layer 2: Input Features = {encoder_ffn_layer2.in_features}, Output Features = {encoder_ffn_layer2.out_features}")

    # Print dimensions for the decoder feed-forward network
    if transformer.decoder is not None and len(transformer.decoder.layers) > 0:
        decoder_ffn_layer1 = transformer.decoder.layers[0].linear1
        decoder_ffn_layer2 = transformer.decoder.layers[0].linear2
        print("Decoder Feedforward Network:")
        print(f"  Layer 1: Input Features = {decoder_ffn_layer1.in_features}, Output Features = {decoder_ffn_layer1.out_features}")
        print(f"  Layer 2: Input Features = {decoder_ffn_layer2.in_features}, Output Features = {decoder_ffn_layer2.out_features}")


transformer_model = Transformer()
print_feedforward_dimensions(transformer_model)

Encoder Feedforward Network:
  Layer 1: Input Features = 512, Output Features = 2048
  Layer 2: Input Features = 2048, Output Features = 512
Decoder Feedforward Network:
  Layer 1: Input Features = 512, Output Features = 2048
  Layer 2: Input Features = 2048, Output Features = 512


### Counting Feed-Forward Number of Parameters

In [8]:
def count_ffn_parameters(transformer):
    total_params = 0
    # Assume each encoder and decoder layer has identical feedforward networks, so we just need to calculate for one and multiply
    if transformer.encoder is not None and len(transformer.encoder.layers) > 0:
        # Calculate for one layer in the encoder
        encoder_ffn_layer1 = transformer.encoder.layers[0].linear1
        encoder_ffn_layer2 = transformer.encoder.layers[0].linear2
        # Layer 1 parameters
        params_layer1 = encoder_ffn_layer1.weight.numel() + encoder_ffn_layer1.bias.numel()
        # Layer 2 parameters
        params_layer2 = encoder_ffn_layer2.weight.numel() + encoder_ffn_layer2.bias.numel()
        # Total parameters for one encoder layer's feedforward network
        total_params_per_layer = params_layer1 + params_layer2
    
    # Multiply by the total number of encoder and decoder layers
    num_layers = len(transformer.encoder.layers) + len(transformer.decoder.layers)
    # Total parameters in all feedforward networks
    total_params = total_params_per_layer * num_layers

    return total_params

total_ffn_params = count_ffn_parameters(model)
print("Total parameters in feedforward networks:", total_ffn_params)

Total parameters in feedforward networks: 25196544


### Counting All Parameters in the Transformer Model

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total parameters:", count_parameters(model))

Total parameters: 44140544


In [ ]:

train_loader = DataLoader(iwslt_dataset['train'], batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(iwslt_dataset['validation'], batch_size=BATCH_SIZE,collate_fn=collate_fn)


optimizer = torch.optim.Adam(list(model.parameters()) + list(output_projection.parameters()), lr=1e-4)

# This is closer to the papers optimizer
# optimizer = torch.optim.Adam(
#     list(model.parameters()) + list(output_projection.parameters()),
#     lr=1e-4,  # This initial learning rate is a placeholder, it will be updated by the scheduler
#     betas=(0.9, 0.98),  # β1 and β2
#     eps=1e-9  # ϵ
# )


num_epochs = 5


for epoch in range(num_epochs):
    for batch in train_loader:
        src_tensor = batch['en']
        tgt_tensor = batch['de']
        
        optimizer.zero_grad()

        src = en_embedding(src_tensor)
        tgt = de_embedding(tgt_tensor)

        # Adjust the sequence length compatibility as before
        if src.shape[1] > tgt.shape[1]:
            src = src[:, :tgt.shape[1], :]
        elif src.shape[1] < tgt.shape[1]:
            pad_size = tgt.shape[1] - src.shape[1]
            src = torch.nn.functional.pad(src, (0, 0, 0, pad_size), value=EN_VOCAB['<pad>'])

        out = model(src, tgt)
        out = output_projection(out)  # Project output to the DE vocabulary size

        # New code for calculating loss, ignoring padding
        target_mask = (tgt_tensor != DE_VOCAB['<pad>']).view(-1)
        loss = torch.nn.functional.cross_entropy(out.view(-1, len(DE_VOCAB)), tgt_tensor.view(-1), reduction='none')
        loss = (loss * target_mask).sum() / target_mask.sum()

        # if torch.backends.mps.is_available():
        #     torch.mps.empty_cache()

        loss.backward()
        
        optimizer.step()

        print(f'Epoch: {epoch}, Loss: {loss.item()}')


    # Save a checkpoint at the end of each epoch
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'output_projection_state_dict': output_projection.state_dict(),  # Correct key for output projection state
        'optimizer_state_dict': optimizer.state_dict(),
        'vocab_en': EN_VOCAB,
        'vocab_de': DE_VOCAB,
        'settings': {
            'd_model': d_model,
            'nhead': 8,
            'num_encoder_layers': 6,
            'num_decoder_layers': 6,
            'dim_feedforward': 2048,
            'output_vocab_size': len(DE_VOCAB)
        }
    }

    torch.save(checkpoint, f'model_ckpt/transformer_checkpoint_epoch{epoch}.pth')


